Import libraries needed

In [19]:
import pandas as pd
import numpy as np

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

!pip install geocoder
import geocoder # import geocoder

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


### Read csv

In [4]:
pa_df = pd.read_csv (r'C:\Users\renat\OneDrive - AUT University\Data Science Courses\IBM Data Science Course\Course 9 Capstone project\physical activity by ny county.csv')
pa_df.head()

,NY County,Physical Inactivity,Confidence interval,County,Rank
0,Albany,20.40%,42.6016 -73.9727,Albany,Rank: 8
1,Allegany,20.20%,42.26 -78.0476,Allegany,Rank: 6
2,Bronx,31.10%,40.8517 -73.8632,Bronx,Rank: 62
3,Broome,26.00%,42.2073 -75.9218,Broome,Rank: 51
4,Cattaraugus,26.30%,42.2486 -78.6784,Cattaraugus,Rank: 53


In [5]:
pa_df.shape

(62, 5)

### Clean PA data

Drop columns that are not needed

In [10]:
pa_df.drop(pa_df.columns[2:], axis=1, inplace=True)
pa_df.head()


,NY County,Physical Inactivity
0,Albany,20.40%
1,Allegany,20.20%
2,Bronx,31.10%
3,Broome,26.00%
4,Cattaraugus,26.30%


In [11]:
pa_df.shape

(62, 2)

In [ ]:
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

- https://opencagedata.com/dashboard#api-keys
- https://nbviewer.jupyter.org/gist/sbscully/c024c31c49ecfea8579f20bf025b50db/opencage-jupyter-notebook.ipynb
- https://amaral.northwestern.edu/blog/getting-long-lat-list-cities

Define Foursquare credentials

In [21]:
CLIENT_ID = 'KLQ3QKOXT34YAZOCUGPDS4EN0XDZJI5J0QDVM2HXKSKPFW1Z' # your Foursquare ID
CLIENT_SECRET = 'HBOI4VNNCH1PQ01HRFPF33OFJEPJX22YYWOHHJ54IZMIIALH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KLQ3QKOXT34YAZOCUGPDS4EN0XDZJI5J0QDVM2HXKSKPFW1Z
CLIENT_SECRET:HBOI4VNNCH1PQ01HRFPF33OFJEPJX22YYWOHHJ54IZMIIALH


In [22]:
#pa_df.loc[1, 'NY County']


'Allegany, NY'

In [103]:
state_latitude = pa_df.loc[1, 'Latitude'] # neighborhood latitude value
state_longitude = pa_df.loc[1, 'Longitude'] # neighborhood longitude value

#state_name = pa_df6.loc[1, 'State'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(state_name, 
                                                               state_latitude, 
                                                               state_longitude))

Latitude and longitude values of Alabama are 32.840571122, -86.631860762.


In [23]:
county_name = '{}, NY'.format(pa_df.loc[0, 'NY County'])
limit=500
#https://api.foursquare.com/v2/venues/search?categoryId=50aa9e094b90af0d42d5de0d,530e33ccbcbc57f1066bbff3,530e33ccbcbc57f1066bbff9,4f2a25ac4b909258e854f55f&intent=browse&client_id={{CLIENT_ID}}&radius=30000&v=20171031&ll=51.51453,-0.12574&client_secret={{CLIENT_SECRET}}&limit=30

#url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}'.format(
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    county_name,
    limit, 
    )
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=KLQ3QKOXT34YAZOCUGPDS4EN0XDZJI5J0QDVM2HXKSKPFW1Z&client_secret=HBOI4VNNCH1PQ01HRFPF33OFJEPJX22YYWOHHJ54IZMIIALH&v=20180605&near=Albany, NY&limit=500'

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5efc08daf5be0232e5b87789'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'geocode': {'what': '',
   'where': 'albany ny',
   'center': {'lat': 42.65258, 'lng': -73.75623},
   'displayString': 'Albany, NY, United States',
   'cc': 'US',
   'geometry': {'bounds': {'ne': {'lat': 42.722379, 'lng': -73.723263},
     'sw': {'lat': 42.61379, 'lng': -73.898099}}},
   'slug': 'albany-new-york',
   'longId': '72057594043034770'},
  'headerLocation': 'Albany',
  'headerFullLocation': 'Albany',
  'headerLocationGranularity': 'city',
  'totalResults': 125,
  'suggestedBounds': {'ne': {'lat': 42.73007885465835,
    'lng': -73.73509176270888},
   'sw': {'lat': 42.634038852174804, 'lng': -73.87260118120467}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This 

In [25]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Emack & Bolio's,Ice Cream Shop,42.641571,-73.780116
1,Ragonese Imports,Sandwich Place,42.654355,-73.794158
2,The Crossings of Colonie,Park,42.717538,-73.792892
3,Druthers Brewing Co.,Brewery,42.665992,-73.741342
4,Risotto,Mediterranean Restaurant,42.675019,-73.748933
...,...,...,...,...
95,Starbucks,Coffee Shop,42.678410,-73.830360
96,Stewart's Shops,Convenience Store,42.697850,-73.791502
97,Regal Colonie Center & RPX,Movie Theater,42.710961,-73.816657
98,Earthworld Comics,Bookstore,42.670411,-73.778732


In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [28]:
LIMIT = 500
def getvenues(counties):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][1]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][1]['name']) for v in results])

    venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    venues.columns = ['NY_County', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(venues)

In [29]:
#run function and create new dataframe
NY_venues = getvenues(counties=pa_df['NY County'])

NameError: name 'pa_df6' is not defined